PII detection

In [3]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware
from langchain_ollama import ChatOllama
from langchain.messages import HumanMessage,AIMessage,SystemMessage
model=ChatOllama(model="gpt-oss:120b-cloud")
agent = create_agent(
    model=model,
    tools=[],
    middleware=[
        # Redact emails in user input before sending to model
        PIIMiddleware(
            "email",
            strategy="redact",
            apply_to_input=True,
        ),
        # Mask credit cards in user input
        PIIMiddleware(
            "credit_card",
            strategy="mask",
            apply_to_input=True,
        ),
        # Block API keys - raise error if detected
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
            apply_to_input=True,
        ),
    ],
)
    
# When user provides PII, it will be handled according to the strategy
result = agent.invoke({
    "messages": [HumanMessage("My email is john.doe@example.com and card is 4532-1234-5678-9010"),HumanMessage("Could you please tell me my email address?")]
})
result

{'messages': [HumanMessage(content='My email is john.doe@example.com and card is 4532-1234-5678-9010', additional_kwargs={}, response_metadata={}, id='66eae48b-a3a4-461d-9b9c-e5ab120fb54e'),
  HumanMessage(content='Could you please tell me my email address?', additional_kwargs={}, response_metadata={}, id='6635be70-7f5b-475d-8c3b-8493d17ef20c'),
  AIMessage(content='Your email address is **john.doe@example.com**.', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-16T08:16:18.637321411Z', 'done': True, 'done_reason': 'stop', 'total_duration': 974060659, 'load_duration': None, 'prompt_eval_count': 108, 'prompt_eval_duration': None, 'eval_count': 94, 'eval_duration': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'}, id='lc_run--bc8ed5fb-a896-4a2a-a482-95b9fce77bdf-0', usage_metadata={'input_tokens': 108, 'output_tokens': 94, 'total_tokens': 202})]}

human in the loop

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command
from langchain_ollama import ChatOllama
model=ChatOllama(model="gpt-oss:120b-cloud")
agent = create_agent(
    model=model,
    tools=[],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                # Require approval for sensitive operations
                "send_email": True,
                "delete_database": True,
                # Auto-approve safe operations
                "search": False,
            }
        ),
    ],
    # persist the state across interrupts
    checkpointer=InMemorySaver(),

)

# Human-in-the-loop requires a thread ID for persistence
config = {"configurable": {"thread_id": "some_id"}}

# Agent will pause and wait for approval before executing sensitive tools
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Send an email to the team"}]},
    config=config
)

result = agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=config  # Same thread ID to resume the paused conversation
)

In [7]:
result

{'messages': [HumanMessage(content='Send an email to the team', additional_kwargs={}, response_metadata={}, id='c958fc82-276b-4324-b5a6-b247f9090d8a'),
  AIMessage(content='I’m not able to send messages or emails directly, but I can help you draft a clear, effective email that you can copy into your email client and send to the team.  \n\nTo tailor it to your needs, could you let me know a few details?\n\n1. **Purpose / Subject line** – What’s the main reason for the email (e.g., project update, meeting invitation, request for feedback)?\n2. **Key points** – Any specific information, milestones, decisions, or actions you want to include?\n3. **Tone** – Formal, friendly, urgent, celebratory, etc.?\n4. **Recipients** – Is it the whole team, a subset, or a specific role (e.g., developers, marketing)?\n5. **Attachments or links** – Any files or URLs that should be referenced?\n\nOnce I have that information, I can draft the full email for you. If you prefer, I can also give you a generic t

Before agent guardrails


In [ ]:
from typing import Any

from langchain.agents.middleware import before_agent, AgentState, hook_config
from langgraph.runtime import Runtime
from langchain_ollama import ChatOllama
model=ChatOllama(model="gpt-oss:120b-cloud")
banned_keywords = ["hack", "exploit", "malware"]

@before_agent(can_jump_to=["end"])
def content_filter(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Deterministic guardrail: Block requests containing banned keywords."""
    # Get the first user message
    if not state["messages"]:
        return None

    first_message = state["messages"][0]
    if first_message.type != "human":
        return None

    content = first_message.content.lower()

    # Check for banned keywords
    for keyword in banned_keywords:
        if keyword in content:
            # Block execution before any processing
            return {
                "messages": [{
                    "role": "assistant",
                    "content": "I cannot process requests containing inappropriate content. Please rephrase your request."
                }],
                "jump_to": "end"
            }

    return None

# Use the custom guardrail
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[],
    middleware=[content_filter],
)

# This request will be blocked before any processing
result = agent.invoke({
    "messages": [{"role": "user", "content": "How do I hack into a database?"}]
})

after agent guardrails

In [ ]:
from langchain.agents.middleware import after_agent, AgentState, hook_config
from langgraph.runtime import Runtime
from langchain_core.messages import AIMessage
from langchain.chat_models import init_chat_model
from typing import Any
from langchain_ollama import ChatOllama
model=ChatOllama(model="gpt-oss:120b-cloud")
@after_agent(can_jump_to=["end"])
def safety_guardrail(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Model-based guardrail: Use an LLM to evaluate response safety."""
    # Get the final AI response
    if not state["messages"]:
        return None

    last_message = state["messages"][-1]
    if not isinstance(last_message, AIMessage):
        return None

    # Use a model to evaluate safety
    safety_prompt = f"""Evaluate if this response is safe and appropriate.
    Respond with only 'SAFE' or 'UNSAFE'.

    Response: {last_message.content}"""

    result = model.invoke([{"role": "user", "content": safety_prompt}])

    if "UNSAFE" in result.content:
        return {
            "messages": [{
                "role": "assistant",
                "content": "I cannot provide that response. Please rephrase your request."
            }],
            "jump_to": "end"
        }

    return None

# Use the safety guardrail
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[],
    middleware=[safety_guardrail],
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "How do I make explosives?"}]
})

combine multiple guardrails

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware, HumanInTheLoopMiddleware,

agent = create_agent(
    model="openai:gpt-4o",
    tools=[],
    middleware=[
        # Layer 1: Deterministic input filter (before agent)
        #ContentFilterMiddleware(banned_keywords=["hack", "exploit"]),

        # Layer 2: PII protection (before and after model)
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("email", strategy="redact", apply_to_output=True),

        # Layer 3: Human approval for sensitive tools
        HumanInTheLoopMiddleware(interrupt_on={"send_email": True}),

        # Layer 4: Model-based safety check (after agent)
        #SafetyGuardrailMiddleware(),
    ],
)